In [1]:
import pandas as pd
import os
import ast
from tqdm import tqdm

In [2]:
FOLDER_PATH = 'partitioned/'

In [3]:
result_df = pd.DataFrame({"artifact_name": [], "transitions":[], "image_link":[]})

In [4]:
for filename in os.listdir(FOLDER_PATH):
    if filename.endswith(".csv"): 
        full_path = os.path.join(FOLDER_PATH, filename)
        read = pd.read_csv(full_path, index_col=False)
        result_df = pd.concat([result_df, read])
        

In [5]:
cd ..

C:\Repos\sparkyShinyIndianaJones


In [6]:
from utils.time_utils import *
from utils.location_utils import *

In [7]:
result_df.head()

,artifact_name,transitions,image_link
0,"Aphrodite, known as the ""Venus de Milo""","{1821: {'lat': 48.8606111, 'lng': 2.337644}}{}...",https://www.louvre.fr/sites/default/files/imag...
1,"The ""Regent"" Diamond","{1887: {'lat': 48.8606111, 'lng': 2.337644}}{}...",https://www.louvre.fr/sites/default/files/imag...
2,Frise des archers,"{1885 - 1886: {'lat': 48.8606111, 'lng': 2.337...",https://www.louvre.fr/sites/default/files/imag...
3,"Horse Restrained by a Groom, called ""Horse of ...","{1802: {'lat': 48.8606111, 'lng': 2.337644}}{1...",https://www.louvre.fr/sites/default/files/imag...
4,"July 28. Liberty Leading the People (July 28, ...","{1983: {'lat': 48.8606111, 'lng': 2.337644}}{1...",https://www.louvre.fr/sites/default/files/imag...


In [8]:
def prepare_transition_string(x):
    x = x.replace(', {', '{')
    to_delete = ['[', ']', '{}']
    for ch in to_delete:
        x = x.replace(ch, '')
    x = x.replace('}{', '}\t{')
    x = x.split('\t')
    return x

In [9]:
def get_date(tran):
    date_str = tran.split(':')[0][1:].strip()
    try:
        return parse_date_year(date_str)
    except:
        return None

In [10]:
def get_location(tran):
    try:
        try:
            dic = ast.literal_eval(tran)
            lt_ln = list(dic.values())[0]
            return lt_ln['lat'], lt_ln['lng']
        except:
            loc = tran.split(':')[1][:-1].strip()
            lt_ln = get_lat_lon(loc)
            if not lt_ln:
                return None, None
            return lt_ln['lat'], lt_ln['lng']
    except:
        return None, None

In [11]:
flattened_df = pd.DataFrame({"artifact_name": [], "year":[], "lat":[], "lon":[], "image_link":[]})

In [12]:
for index, row in tqdm(result_df.iterrows()):
    transitions = prepare_transition_string(row['transitions'])
    for tran in transitions:
        date = get_date(tran)
        if not date:
            continue
        lat, lon = get_location(tran)
        if not lat:
            continue
        flattened_df = flattened_df.append({
            "artifact_name": row['artifact_name'],
            "year": date,
            "lat" : lat,
            "lon" : lon,
            "image_link" : row['image_link']
        }, ignore_index = True)

460it [00:01, 297.48it/s]


In [13]:
flattened_df.head()

,artifact_name,year,lat,lon,image_link
0,"Aphrodite, known as the ""Venus de Milo""",1821.0,48.860611,2.337644,https://www.louvre.fr/sites/default/files/imag...
1,"The ""Regent"" Diamond",1887.0,48.860611,2.337644,https://www.louvre.fr/sites/default/files/imag...
2,"Horse Restrained by a Groom, called ""Horse of ...",1802.0,48.860611,2.337644,https://www.louvre.fr/sites/default/files/imag...
3,"July 28. Liberty Leading the People (July 28, ...",1983.0,48.860611,2.337644,https://www.louvre.fr/sites/default/files/imag...
4,Diptych panel in five parts: the Emperor Trium...,1899.0,48.860611,2.337644,https://www.louvre.fr/sites/default/files/imag...


In [14]:
SAVING_PATH = 'visualization/data/for_visual.csv'

In [15]:
flattened_df.to_csv(SAVING_PATH, index=False)